Now that you know how to create an attention mechanism, we will use it in this notebook to implement an attention transformer in PyTorch. The format of the notebook will be roughly the same as previous notebooks (just fill in when it asks for your_implementation). 

# Setup

We first have to import the PyTorch libraries to build the Transformer model.

In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F
# Can import from attention module in week4

Here we define the hyperparameters for the model and training.

In [2]:
### Hyperparameters
batch_size = 16 # how many independent sequences will we process in parallel?
block_size = 32 # what is the maximum context length for predictions?
max_iters = 5000 # how many iterations do we want to train our model for
eval_interval = 100 # at which iterations do we perform our evaluation
learning_rate = 1e-3 # how much do we want to optimize our weights at each step
device = 'cuda' if torch.cuda.is_available() else 'cpu' # determines device we run the tensor on
eval_iters = 200 # how many evaluation intervals do we use to get the loss average
n_embd = 64 # dimension of embeddings for our input
n_head = 4 # number of attention heads working in parallel
n_layer = 4 # number of layers in our attention head that our input goes through
dropout = 0.0 # dropout probability aka probability that a weight turns to 0

Here we load in our dataset to be used by the Transformer Model we are creating.

In [3]:
### Preparing Data
torch.manual_seed(1337)

# wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

These are miscellaneous functions that will be used in our model training and evaluation. 

The first function gets the batches for out dataset. For our data, we often process it in batches. Each batch is a set size of data taken randomly from the dataset. Each training epoch operates on one batch. The input is always what is ahead in the sequence and the target is the next character.

The second function estimates the loss for our model using a simple training loop with gradient calculation.

In [4]:
### Miscellaneous Functions
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    # estimate the average loss for each data split for evaluation
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

# LayerNorm

Now we build LayerNorm from scratch. 

![The equation we use for LayerNorm](https://miro.medium.com/v2/resize:fit:1040/0*qN-QGSHiY85obQfj)

Basically, LayerNorm takes the mean and variance of the input x and uses it to normalize the inputs. Following the Central Limit Theorem, we normalize the inputs so that they have a mean of 0 and a variance of 1 by subtracting the mean and dividing by the standard deviation (square root of variance). This allows the inputs to follow a standard normal distribution preventing the data from getting extraneous or outlier values that can cause exploding or vanishing gradients, allowing for more stable training. In order to make sure that we don't divide by 0, we add epsilon, a small value, to the denominator.

Next, we have the shift and scale parameters. Gamma is the scale parameter which is basically the variance of the normalized distribution. You can think of it as how much the normal distribution is stretched from the standard normal (like how large the range of common values is). Beta is the shift parameter which is basically the mean of the normalized distribution. This is where the normal distribution is centered. These two parameters are learned so you can adjust them to the data that is used. 

In [5]:
class LayerNorm(nn.Module):
    "Construct a layernorm module (See citation for details)."

    def __init__(self, features, eps=1e-6):
        super(LayerNorm, self).__init__()
        # initialize the parameters of the LayerNorm equation
        self.a_2 = nn.Parameter(torch.ones(features))
        self.b_2 = nn.Parameter(torch.zeros(features))
        self.eps = eps

    def forward(self, x):
        # implement LayerNorm based on the equation above
        mean = x.mean(-1, keepdim=True)
        std = x.std(-1, keepdim=True)
        return self.a_2 * (x - mean) / (std + self.eps) + self.b_2

In [6]:
# class FeedFoward(nn.Module):
#     """ a simple linear layer followed by a non-linearity """

#     def __init__(self, n_embd):
#         super().__init__()
#         self.net = nn.Sequential(
#             nn.Linear(n_embd, 4 * n_embd),
#             nn.ReLU(),
#             nn.Linear(4 * n_embd, n_embd),
#             nn.Dropout(dropout),
#         )

#     def forward(self, x):
#         return self.net(x)

# Attention

Next, lets build the attention mechanism for our Transformer. This is the core component that makes Transformers so powerful.

First, we will define the attention head where our attention mechanism will take place. In it we initialize the key, query, and value projection layers where we project the inputs into another subspace (think of it as another dimension or embedding of storing information on the input semantics). These projection layers are learned and can be adjusted. We also defined something called "tril" which we will go into more in just a second. Finally, there is dropout a regularization technique to prevent overfitting and allow the model to learn more effectively.

In the attention mechanism, we first need to get our key, query, and value embeddings. We do this by throwing the input through linear layers to map them to another information filled subspace. These embeddings are very important. For example, lets say your input is a sentence. Each sentence is broken up into tokens (typically one word) and these tokens are represented by embeddings. The projection layers map these embeddings into another subspace as vectors (key, value, and query). The magnitude and direction or what defines these vectors encode information about the token they are correlated to. We will use this information later on to determine the meaning of the input sentence in our attention mechanism. 

We can use Week 4's attention notebook and implementation here.

![Here is the attention mechanism equation](https://miro.medium.com/v2/resize:fit:1400/0*4L90D4iDB_R1Uljs)

In [7]:
class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

# Multi Head Attention

After developing our attention mechanism, the engine of the Transformer model, we need to add a few things to complete the architecture. One of them is MultiHeadAttention. The idea of MultiHeadAttention is to allow for multiple attention mechanisms to happen in parallel allowing for more information and semantic parsing. In longer sequences with longer contexts, this extra information is very useful in order to maximize accuracy and efficiency. Each attention head in a MultiHeadAttention block can learn a separate function and thus retain different things about the input. For example, with a sentence, one head can learn about what the subject is doing and another head can learn about how the subject looks.

![Diagram of MultiHeadAttention](https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcR2H71D22diHDQKf6STcbHbRgvdynJ_c0RZZA&s)

In [8]:
class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        # self.proj = nn.Linear(n_embd, n_embd)
        # self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # print(x.shape)
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        # print(out.shape)
        # out = self.dropout(self.proj(out))
        return out

Next, we need to put it all together into a Transformer Block. First, we initialize the MultiHeadAttention block to be used in the Transformer. For each head, the subspace that they work in is the size of the embedding divided by the number of heads. The more heads can lead to more parallel attention mechanisms at once but also force the embedding space they work in to be smaller, leading to more limited information. We also initialize a layer norn as regularization for the output of the MultiHeadAttention block. In a normal Transformer block there would also be a feed forward but we will cover that in more detail later.

The forward function just uses the initialized MultiHeadAttention on the input and throws the output of that throught the LayerNorm.

In [9]:
class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        # initialize the components of a attention only Block
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ln1 = nn.LayerNorm(n_embd)

    def forward(self, x):
        # put the input throught the intialized components
        x = x + self.sa(self.ln1(x))
        return x

# BigramAttentionModel

Finally, we put the blocks together to create the BigramAttentionModel. First we have a token_embedding_table which basically embeds the input tokens into embeddings based on a table. These embeddings are numerical representations of the input, vectors of size n_embd that contain information of the input they are representing. 

![Here is a diagram of embeddings. As you can see words are mapped to vectors where more related words have more similar vectors](https://arize.com/wp-content/uploads/2022/06/blog-king-queen-embeddings.jpg)

Next, we have the position_embedding_table which is used to encode the position of each token in the input. This is another lookup table that takes the position of a token in the sequence and encodes it in a numerical vector of the same size as the input embeddings. This is done so that we can add the positional embeddings to the input embeddings. This allows the model to have awareness of where in a sentence a specific token is and the relative distances between different tokens. 

![In a typical Transformer, we use sin and cos functions to create positional encodings. However, we are just using a table for this application. Here is how positional encodings are used within a LLM.](https://miro.medium.com/v2/resize:fit:1400/0*oiP-eu8BmJx5SVp7.png)

Lastly, we have the actual transformer blocks which is the workhorse of the model. These blocks contain the multi headed attention mechanism. 

We finally have a linear layer to map the embeddings and the information they carry to the vocab_size so they can be decoded character by character into natural language. LayerNorm is used once again to regularize the output of the blocks. 

The forward function uses all the initialized modules on the input in order and then if a target is provided, helps compute the loss. We use cross entropy between the predicted next character logit (which is basically the probability distribution of the next character) and the target next character to calculate the loss. In order to make it compatible with the cross entropy function we need to reshape the tensor so that the dimensions match. The dimensions of the two tensors are provided in the comments.

We also have a generate function for inference. The idea of this function is to generate max_new_tokens of characters within a natural language passage using the Bigram Language Transformer Model. 

## Depth of Model

A major hyperparameter to consider is depth of the model. To create a deeper model, we can add more Transformer Blocks to the model (increase what n_layer is). But what is the advantage of that?

In lots of scenarios, stacking layers and having more parameters can lead to greater results as evidenced by the increasing size of LLMs and their growing capabilities. However, just stacking more blocks isn't necessarily easy.

More blocks means more compute and that means more necessary GPUs and physical hardware to support the machine. There's a reason why only big companies with lots of money can create the best models in the world. There's also the problem of unstable training. With bigger models, they become harder to train due to unstable gradients. Gradient calculation starts from the lowest layer and becomes smaller and smaller as you go through the model. This can lead to vanishing gradients or increasingly slow training.

In [10]:
class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        logits = self.blocks(x) # (B,T,C)
        x = self.ln_f(logits) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C) # (B*T, C)
            targets = targets.view(B*T) # (B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

# Training Module

This is the training module. Here we intialize the model, print the parameters, then have a typical training loop for the Transformer Model with evaluation intervals. 

In [11]:
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    # estimate the average loss for each data split for evaluation
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

### Model Training and Initialization
model = BigramLanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

0.060225 M parameters
step 0: train loss 4.3867, val loss 4.3800
step 100: train loss 3.0054, val loss 3.0246
step 200: train loss 2.6671, val loss 2.6851
step 300: train loss 2.5606, val loss 2.5675
step 400: train loss 2.4906, val loss 2.5020
step 500: train loss 2.4404, val loss 2.4377
step 600: train loss 2.4089, val loss 2.4043
step 700: train loss 2.3601, val loss 2.3743
step 800: train loss 2.3442, val loss 2.3313
step 900: train loss 2.3211, val loss 2.3368
step 1000: train loss 2.2852, val loss 2.2934
step 1100: train loss 2.2754, val loss 2.2840
step 1200: train loss 2.2501, val loss 2.2645
step 1300: train loss 2.2343, val loss 2.2500
step 1400: train loss 2.2156, val loss 2.2359
step 1500: train loss 2.2058, val loss 2.2278
step 1600: train loss 2.1896, val loss 2.2114
step 1700: train loss 2.1615, val loss 2.1906
step 1800: train loss 2.1630, val loss 2.1883
step 1900: train loss 2.1366, val loss 2.1748
step 2000: train loss 2.1314, val loss 2.1739
step 2100: train loss 2.

This code uses the model to generate a shakespeare passage since the dataset we gave the model to train on is shakespeare passages.

In [12]:
# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=2000)[0].tolist()))


WAll becordy.

Siad Sir:
Well madisen bobe to takegr-and me?
Leants art that usquely.

PAUMEXENS:
Aendwice my feanstacom oroug
Youts, tof is heir this now
Whineg is eest hein latistlidiov the and the now theeran of likind teal.
His me nervey preperness hew yean'st the nroupetel'd gond:
I should thak
Aon in on her eiiby own thesourise
And hime sto-on

ADWITHARD:
O hanterup af so;
Ang hinke me tey aled
At there shee my of.

HKING ERY:
Now is wards. Wich EDn shovein cour aaranny ire to-chan the!

JOUnEET:
Pke mary thou come blest so a thense.--
KING RICHARD Areand the mest hes the wetcend,
Thou cous somy shoutcas lort
IEn shouth peakes.

LAUCIPTABUS:
To to-didpasst
Whith in meent some so nestas or fether,
Thy, com loovers! I sumbuside he nees oug
For thirbeake begrerveinn's an so! with selfore,
Whoscellosse's is andcessinge, his I ither porven'end.

Centey sevencees: timed the now the summpelip!

GLIS what that the but speire ollke, of so hat there, drewatch time to ther foru;
you genton

What do you notice about the passage?

Fill in here: